# Descripción
El data set cuenta con los datos de calidad de red movil de las compañias de telecomunicaciones de Chile correspondientes al periodo de Noviembre del 2018 y sus indicadores principales.

# Objetivo
- Evaluar el desempeño de las empresas 
- Identificar estaciones problematicas
- Desempeño por region


In [1]:
import pandas as pd
import math as mt
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns

In [2]:
df = pd.read_excel('Calidad_red_movil_201811_WEB.xlsx',sheet_name='Datos')

In [3]:
df.head()

,COD_EMPRESA,PERIODO,ESTACIONBASE,RANGOHORARIO,COMUNA,REGION,DESCRIPCIONCOMUNA,TIPOEMPLAZAMIENTO,LLAMADASEXITO,INTENTOLLAMADA,LLAMADAINTERRUP,TIEMPOINTERRUP,PEE,PFE
0,Movistar,201811,HRAG1U01,1213,1404,1,Huara,R,2346,2349,83,0,0.998723,0.964621
1,Movistar,201811,DARCNA,1213,1101,1,Iquique,R,174,174,11,0,1.000000,0.936782
2,Movistar,201811,MINIMU01,1213,1404,1,Huara,R,30,30,1,0,1.000000,0.966667
3,Movistar,201811,PASUF6,1213,1401,1,Pozo Almonte,R,3168,3168,31,0,1.000000,0.990215
4,Movistar,201811,CMIQU,1213,1101,1,Iquique,U,2703,2703,87,0,1.000000,0.967814


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19967 entries, 0 to 19966
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COD_EMPRESA        19967 non-null  object 
 1   PERIODO            19967 non-null  int64  
 2   ESTACIONBASE       19967 non-null  object 
 3   RANGOHORARIO       19967 non-null  int64  
 4   COMUNA             19967 non-null  int64  
 5   REGION             19967 non-null  int64  
 6   DESCRIPCIONCOMUNA  19967 non-null  object 
 7   TIPOEMPLAZAMIENTO  19967 non-null  object 
 8   LLAMADASEXITO      19967 non-null  int64  
 9   INTENTOLLAMADA     19967 non-null  int64  
 10  LLAMADAINTERRUP    19967 non-null  int64  
 11  TIEMPOINTERRUP     19967 non-null  int64  
 12  PEE                19967 non-null  float64
 13  PFE                19967 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 2.1+ MB


In [5]:
df.isna().sum()

COD_EMPRESA          0
PERIODO              0
ESTACIONBASE         0
RANGOHORARIO         0
COMUNA               0
REGION               0
DESCRIPCIONCOMUNA    0
TIPOEMPLAZAMIENTO    0
LLAMADASEXITO        0
INTENTOLLAMADA       0
LLAMADAINTERRUP      0
TIEMPOINTERRUP       0
PEE                  0
PFE                  0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
print(df['PERIODO'].unique())
print(df['RANGOHORARIO'].unique())

[201811]
[1213]


- El data set cuenta con 14 columnas
- El data set no posee registros vacios ni duplicados
- Las columnas PERIODO y RANGOHORARIO no son tan representativas ya que todos los registros estan en el mismo rango temporal en ambos casos
- La columna COMUNA es de tipo  int lo cual no es necesario ya que este es un numero identificador de la comuna por lo que realizar alguna operacion no es significativo 

# Pre procesamiento

In [10]:
df_red = df.drop(['PERIODO','RANGOHORARIO'],axis=1)
df_red

,COD_EMPRESA,ESTACIONBASE,COMUNA,REGION,DESCRIPCIONCOMUNA,TIPOEMPLAZAMIENTO,LLAMADASEXITO,INTENTOLLAMADA,LLAMADAINTERRUP,TIEMPOINTERRUP,PEE,PFE
0,Movistar,HRAG1U01,1404,1,Huara,R,2346,2349,83,0,0.998723,0.964621
1,Movistar,DARCNA,1101,1,Iquique,R,174,174,11,0,1.000000,0.936782
2,Movistar,MINIMU01,1404,1,Huara,R,30,30,1,0,1.000000,0.966667
3,Movistar,PASUF6,1401,1,Pozo Almonte,R,3168,3168,31,0,1.000000,0.990215
4,Movistar,CMIQU,1101,1,Iquique,U,2703,2703,87,0,1.000000,0.967814
...,...,...,...,...,...,...,...,...,...,...,...,...
19962,Wom,ARA_3G_008,9101,9,Temuco,U,20479,20489,15,0,0.999512,0.999268
19963,Wom,ARA_3G_011,9101,9,Temuco,U,29019,29027,74,0,0.999724,0.997450
19964,Wom,ARA_3G_017,9210,9,Traiguén,R,9531,9553,217,0,0.997697,0.977232
19965,Wom,ARA_in_3G_006,9101,9,Temuco,U,3137,3137,0,45,1.000000,1.000000


In [11]:
df_red['COMUNA']=df_red['COMUNA'].astype('object')
df_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19967 entries, 0 to 19966
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COD_EMPRESA        19967 non-null  object 
 1   ESTACIONBASE       19967 non-null  object 
 2   COMUNA             19967 non-null  object 
 3   REGION             19967 non-null  int64  
 4   DESCRIPCIONCOMUNA  19967 non-null  object 
 5   TIPOEMPLAZAMIENTO  19967 non-null  object 
 6   LLAMADASEXITO      19967 non-null  int64  
 7   INTENTOLLAMADA     19967 non-null  int64  
 8   LLAMADAINTERRUP    19967 non-null  int64  
 9   TIEMPOINTERRUP     19967 non-null  int64  
 10  PEE                19967 non-null  float64
 11  PFE                19967 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


In [20]:
def pivot (index,column,value,func):
    df=df_red.pivot_table(index=index,columns=column,values=value,aggfunc=(func))
    return df

df_llamadas_ex =  pivot('DESCRIPCIONCOMUNA','COD_EMPRESA','LLAMADASEXITO','sum')

df_llamadas_ex

COD_EMPRESA,Claro,Entel,Movistar,Wom
DESCRIPCIONCOMUNA,,,,
Aisén,51016.0,136562.0,64667.0,8793.0
Algarrobo,112291.0,109483.0,130753.0,26793.0
Alhué,12605.0,34897.0,10051.0,22767.0
Alto Biobío,11068.0,6273.0,9529.0,NaN
Alto Del Carmen,8376.0,15646.0,28932.0,1746.0
...,...,...,...,...
Yumbel,42958.0,87369.0,119849.0,29565.0
Yungay,82353.0,98073.0,82915.0,29750.0
Zapallar,51769.0,57465.0,62165.0,28275.0


In [ ]:
df_intento_llamada = pivot('DESCRIPCIONCOMUNA','COD_EMPRESA','INTENTOLLAMADA','sum')

df_intento_llamada


COD_EMPRESA
Claro        40.0
Entel       920.0
Movistar    128.0
Wom         759.0
dtype: float64
COD_EMPRESA
Claro        Río Verde
Entel       Lago Verde
Movistar      Timaukel
Wom          Lonquimay
dtype: object


In [27]:
max_intento = df_intento_llamada.max()
index_max_intento = df_intento_llamada.idxmax()

df_max_intento_llamada = pd.DataFrame({'max':max_intento,'empresa':index_max_intento})
df_max_intento_llamada

,max,empresa
COD_EMPRESA,,
Claro,3777881.0,Santiago
Entel,6686865.0,Santiago
Movistar,4842720.0,Santiago
Wom,1368674.0,Santiago


In [22]:
df_llamada_interrumpida = pivot('DESCRIPCIONCOMUNA','COD_EMPRESA','LLAMADAINTERRUP','sum')


df_llamada_interrumpida

COD_EMPRESA,Claro,Entel,Movistar,Wom
DESCRIPCIONCOMUNA,,,,
Aisén,208.0,693.0,343.0,16.0
Algarrobo,253.0,553.0,809.0,28.0
Alhué,129.0,127.0,82.0,32.0
Alto Biobío,28.0,96.0,119.0,NaN
Alto Del Carmen,34.0,96.0,290.0,28.0
...,...,...,...,...
Yumbel,449.0,464.0,1586.0,235.0
Yungay,397.0,486.0,820.0,121.0
Zapallar,153.0,242.0,386.0,14.0
